In [1]:
# Import necessary modules
import sys
import os

# Append the directory of your script to the Python system path
# If your notebook is in the 'research' directory and your script is in the 'scripts' directory, 
# you will go up one level from 'research' and then into 'scripts'
sys.path.append(os.path.abspath('../scripts'))

# Now you can import your module
from child_fatality_scrape import * # replace 'your_function_name' with actual function names


In [5]:
import fitz  # PyMuPDF
import re

# Function to count dates in the "G." section of a PDF
def count_dates_in_g_section(pdf_data):
    # Open the PDF from the binary data
    doc = fitz.open(stream=pdf_data, filetype="pdf")
    text = ""
    # Extract text from each page
    for page in doc:
        text += page.get_text()
    doc.close()
    
    # Find the section starting with "G." and ending before "H." or the end of the document
    section_g = re.search(r'(?<=G\.).+?(?=H\.|\Z)', text, re.DOTALL)
    if not section_g:
        return 0  # If "G." section is not found, return 0

    section_g_text = section_g.group(0)
    # print(section_g_text)
    # Define a regex pattern for dates (MM/DD/YYYY or MM/DD/YY)
    date_pattern = re.compile(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b')
    # Find all dates in the "G." section
    dates = date_pattern.findall(section_g_text)

    # kevin intervention, ChatGPT cannot see multi-line 0400 12/03/2021 date from footer, simply subtract 1
    total_dates = len(dates) - 1

    return total_dates

# Paths to the PDF files
pdf_paths = [
    "Public_Disclosure_Fatality_1377353.pdf",
    "1453180_5_2_23.pdf",
    "Public_Disclosure_1510126_F_4.24.23_v2.pdf",
    "2023_04_13_ID_1446073.pdf",
]


def count_pdf_paths(pdf_paths):
    # Dictionary to hold the counts of dates for each PDF
    date_counts = {}

    # Process each PDF file and count the dates in the "G." section
    for pdf_path in pdf_paths:
        # Read the binary data of the PDF file
        with open(pdf_path, "rb") as f:
            pdf_data = f.read()
        # Count the dates in the "G." section
        date_counts[pdf_path] = count_dates_in_g_section(pdf_data)

    return date_counts

count_pdf_paths(pdf_paths)


{'Public_Disclosure_Fatality_1377353.pdf': 12,
 '1453180_5_2_23.pdf': 8,
 'Public_Disclosure_1510126_F_4.24.23_v2.pdf': 0,
 '2023_04_13_ID_1446073.pdf': 7}

In [6]:
# Now you can call your imported functions as needed
pdf_paths = list_files("/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs", 
                       append_base_path=True)

In [7]:
count_pdf_paths(pdf_paths)

{'/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2022-04-15_ID_1488030.pdf': 2,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2022-05-08_ID_1497671.pdf': 0,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2021_12_11_ID_1483068.pdf': 2,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2021_11_10_ID_1489652.pdf': 1,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2021_02_13_ID_1480482.pdf': 0,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2021-04-14_ID1284572(1).pdf': 2,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2021_05_31_ID_1484628.pdf': 0,
 '/Users/kevinkurek/Desktop/github/state_of_nv_child_fatalities/output_files/Clark_pdfs/2023_03_31_ID_1480684.pdf': 2,
 '/Users/kevinkurek/Desktop/github/state_of_nv